### 自动求导

In [ ]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3 * a ** 2 - b ** 2
'''
    需要在 Q.backward() 中显式传入一个 gradient 参数，因为它是向量。
    gradient 是一个与 Q 形状相同的张量，它表示 Q 相对于自身的梯度 dQ/dQ=1
'''
# Q.backward(torch.tensor([1., 1.]))
'''
    等效地，也可以将 Q 聚合为标量并隐式调用反向传播，即 Q.sum().backward()
'''
Q.sum().backward()

'''
    梯度存储在 a.grad 和 b.grad 中
'''
print(a.grad, b.grad)

tensor([12., 18.]) tensor([-12.,  -8.])


### 自定义自动求导

y = k * func1(c * x) + b，其中func1(p)=p^2，dfunc1/dp = 2p

求导

    dk = (c * x) ^ 2

    db = 1

    dc = k * 2(c * x) * x

自定义：
    dfunc1/dp = p

In [4]:
import torch


class fun1(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):  # 必须有 ctx，ctx 表示上下文管理器，一般用来保存在 backward 阶段会用到的 tensor
        ctx.save_for_backward(input)
        out = input ** 2  # 定义反向传播函数 fun1 = input ** 2
        return out

    @staticmethod
    def backward(ctx, grad_output):  # grad_output 为上一层累积的梯度（求导结果）
        input, = ctx.saved_tensors  # 调用 ctx 获得 forward 的 tensor
        return grad_output * input  # 定义 dfun1/dinput = input


fun = fun1.apply
x = torch.tensor([12])
k = torch.full((), 0.5, requires_grad=True)
b = torch.full((), 0.7, requires_grad=True)
c = torch.full((), 0.2, requires_grad=True)
y = k * fun(c * x) + b
y.backward()
print(c.grad)

tensor(14.4000)


定义 $\frac{\partial fun}{\partial p} = 1$

In [5]:
import torch


class fun1(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = input ** 2  # 定义反向传播函数 fun1 = input ** 2
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        return grad_output * 1  # 定义 dfun1/dinput = 1


fun = fun1.apply
x = torch.tensor([12])
k = torch.full((), 0.5, requires_grad=True)
b = torch.full((), 0.7, requires_grad=True)
c = torch.full((), 0.2, requires_grad=True)
y = k * fun(c * x) + b
y.backward()
print(c.grad)

tensor(6.)
